In [ ]:
import pandas as pd 
import pickle
import tensorflow as tf
import torch
import tensorflow.keras.backend as K
import transformers
from transformers import TFCamembertModel
from transformers import CamembertTokenizer
from ipywidgets import IntProgress
"""Probably should try with TFCamemForSequenceClassification"""

In [2]:
# Defining some key variables that will be used later on in the training

MAX_LEN = 2000
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

In [4]:
with open ('../CNN-kim/Data_2labels_CIB4/df_test_clean.pickle','rb') as fichier:
    df_test = pickle.load(fichier)

with open ('../CNN-kim/Data_2labels_CIB4/df_train_clean.pickle','rb') as fichier:
    df = pickle.load(fichier)
    


In [5]:
train_size = 0.8
df_train = df.sample(frac=train_size, random_state=200)
df_dev = df.drop(df_train.index).reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(df_train.shape))
print("TEST Dataset: {}".format(df_dev.shape))

FULL Dataset: (3751492, 5)
TRAIN Dataset: (3001194, 5)
TEST Dataset: (750298, 5)


In [ ]:
print(df_train.head())

In [6]:
def convert_to_camembert_tfdataset(df):
    labels = list(df['cat'])
    
    input_ids = []
    attention_mask = []
    
    for text in df['text']:
        inputs = tokenizer.encode_plus(text,
                                     add_special_tokens=True,
                                     max_length=MAX_LEN,
                                     pad_to_max_length=True,
                                     return_token_type_ids=True,
                                     return_tensors="np",
                                     truncation=True,
                                     )
        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])
        
    return (input_ids,attention_mask,labels)
                                             
        

In [7]:
input_ids,attention_mask,labels = convert_to_camembert_tfdataset(df_train[:200000])



In [11]:
print(input_ids[0].shape)
print(labels[0])



(1, 2000)
2


In [18]:
def _input_fn():
    def generator():
        for ids, mask, l in zip(input_ids, attention_mask, labels):
            yield {"input_id" : ids, "input_mask" : mask }, l
    
    dataset = tf.data.Dataset.from_generator(generator,
                                    output_types= ({"input_id" :tf.int32, "input_mask" : tf.int32},tf.int32),                            
                                    output_shapes= ({"input_id" : (2000), "input_mask" : (2000)}, (1))
                                            )
                            
                                                        
                                            
    dataset = dataset.batch(8)
    return(dataset)

In [ ]:
def _input_fn():
    dataset = tf.data.Dataset.from_tensor_slices(({"input_id" : }))

In [ ]:
training_dataset = convert_to_camembert_tfdataset(df_train[:200000])
dev_dataset = convert_to_camembert_tfdataset(df_dev[:100000])



In [ ]:
print(training_dataset)

In [21]:
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32, name='input_id')
    mask = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32, name='input_mask')
    camembert_model = TFCamembertModel.from_pretrained('camembert-base', from_pt=True)
    
    camembert_layer = camembert_model([ids,mask])[0]
    flattened_layer = tf.keras.layers.Flatten()(camembert_layer)
    dropout_layer = tf.keras.layers.Dropout(0.2)(flattened_layer)
    
    dense_output_layer = tf.keras.layers.Dense(units=435,
                                              activation= 'softmax')(dropout_layer)
    model = tf.keras.models.Model(inputs=[ids,mask], outputs=[dense_output_layer])
    
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss = loss, optimizer=optimizer, metrics=[metric])
    
    return(model)
    
    
    
    

In [23]:
model = build_model()
model.fit([input_ids,attention_mask], labels, epochs=2, batch_size=10)
''' - Bon en gros les données va falloir que je les tokenize ligne par ligne, c'est chiant, et ensuite je pourrais voir mon
entrée à model.fit
regarder si le Sequenceclassification model est bien aussi ou pas. '''


All PyTorch model weights were used when initializing TFCamembertModel.

Some weights or buffers of the PyTorch model TFCamembertModel were not initialized from the TF 2.0 model and are newly initialized: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: in converted code:

    C:\Users\cheim\Anaconda3\envs\camemBert\lib\site-packages\transformers\modeling_tf_roberta.py:231 call  *
        outputs = self.roberta(inputs, **kwargs)
    C:\Users\cheim\Anaconda3\envs\camemBert\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:773 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    C:\Users\cheim\Anaconda3\envs\camemBert\lib\site-packages\transformers\modeling_tf_bert.py:606 call  *
        embedding_output = self.embeddings([input_ids, position_ids, token_type_ids, inputs_embeds], training=training)
    C:\Users\cheim\Anaconda3\envs\camemBert\lib\site-packages\tensorflow_core\python\keras\engine\base_layer.py:822 __call__
        outputs = self.call(cast_inputs, *args, **kwargs)
    C:\Users\cheim\Anaconda3\envs\camemBert\lib\site-packages\transformers\modeling_tf_bert.py:166 call
        return self._embedding(inputs, training=training)
    C:\Users\cheim\Anaconda3\envs\camemBert\lib\site-pac